In [1]:
from pymongo import MongoClient
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
%matplotlib inline

### get route info preped

In [2]:
def dataframe_from_collection(collection):
    '''covnert mongodb dataframe to pandas dataframe'''
    raw_data = collection.find()
    return pd.DataFrame(list(raw_data))

In [3]:
client = MongoClient('mongodb://localhost:27017/')

In [5]:
db = client.routes_updated
route_df = dataframe_from_collection(db.routes)
route_df.head().T

,0,1,2,3,4
Aid,0,0,0,0,0
Alpine,0,0,0,0,0
Boulder,0,0,0,0,0
FA,"Adam Winslow, Bill Coe, Kyle Silverman 3/20/09","Adam Winslow, Ujahn Davisson, Bill Coe, Jim Op...",?,,Unknown
Ice,0,0,0,0,0
Mixed,0,0,0,0,0
Sport,0,0,0,0,0
TR,0,0,1,1,1
Trad,1,1,1,1,1
_id,58d4599740b441389904258b,58d4599740b441389904258c,58d4599740b441389904258d,58d4599740b441389904258e,58d4599740b441389904258f


### make id dataframe

In [6]:
route_df_id = route_df[['name', 'id','average_rating', 'route_url']]

### drop duplicates

In [5]:
# might not need this 
# route_df[route_df.duplicated(subset=['route_url'])].shape

# route_df = route_df.drop_duplicates(subset=['route_url'])

In [7]:
# drop columns
route_df_new = route_df.drop(['uiaa', 'submitted_on',
                            'submitted_by', 'season', 
                            'name', 'id',
                            'average_rating', '_id',
                            'FA', 'route_url'], axis=1)

In [8]:
route_df_new.head()

,Aid,Alpine,Boulder,Ice,Mixed,Sport,TR,Trad,area,grade,height,original_grade,page_views,pitches
0,0,0,0,0,0,0,0,1,(1) The Arena of PleasureaAreas,5.6,41.0,5.6,588,1.0
1,0,0,0,0,0,0,0,1,(1) The Arena of PleasureaAreas,5.5,41.0,5.5,642,1.0
2,0,0,0,0,0,0,1,1,Granite PointaAreas,5.5,35.0,5.5,176,NaN
3,0,0,0,0,0,0,1,1,Barney's RubbleaAreas,5.6,85.0,5.6,1263,1.0
4,0,0,0,0,0,0,1,1,Hammerhead RockaAreas,5.6,30.0,5.6,678,1.0


In [9]:
route_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727 entries, 0 to 3726
Data columns (total 14 columns):
Aid               3727 non-null int64
Alpine            3727 non-null int64
Boulder           3727 non-null int64
Ice               3727 non-null int64
Mixed             3727 non-null int64
Sport             3727 non-null int64
TR                3727 non-null int64
Trad              3727 non-null int64
area              3727 non-null object
grade             3727 non-null object
height            2992 non-null float64
original_grade    3727 non-null object
page_views        3727 non-null int64
pitches           2127 non-null float64
dtypes: float64(2), int64(9), object(3)
memory usage: 407.7+ KB


### fill nulls

In [10]:
route_df_new['height'] = route_df_new['height'].fillna(route_df_new['height'].mean())
route_df_new['pitches'] = route_df_new['pitches'].fillna(route_df_new['pitches'].mean())

### make dummies

In [11]:
df_grades = route_df_new[['grade','original_grade','area']]
# drop linear dependent columns
df_dummies = pd.get_dummies(df_grades).drop(['grade_3rd', 'area_{10} MordoraAreas'], axis=1)

In [12]:
concat_list = [route_df_new.drop(['grade','original_grade', 'area'], axis=1), df_dummies]
x_routes = pd.concat(concat_list, axis=1).fillna(route_df_new.mean())

In [13]:
x_routes.isnull().values.any()

False

In [14]:
x_routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3727 entries, 0 to 3726
Columns: 700 entries, Aid to area_Zig Zag WallaAreas
dtypes: float64(2), int64(9), uint8(689)
memory usage: 2.8 MB


### save dataframe

In [15]:
db = client.utility_matrix
ratings_df = dataframe_from_collection(db.utility_matrix).drop('_id', axis=1)
ratings_df.head()

,rating,route_id,user_id
0,1,0,36
1,1,1,36
2,1,2,394
3,1,2,1972
4,1,2,1625


In [16]:
ratings_df[ratings_df['user_id'] == 560]

,rating,route_id,user_id
1610,3,199,560
2825,4,370,560
5765,3,684,560
8750,4,956,560
11226,4,1200,560
11924,4,1244,560
13621,3,1395,560
16397,3,1705,560
19840,3,2086,560
21128,3,2237,560


In [17]:
# add number of reviews
routes = route_df_id[['id', 'name']].join(x_routes)
routes['num_reviews'] = ratings_df.groupby(['route_id'])['user_id'].transform('count').reset_index().drop('index', axis=1)
routes.head()

,id,name,Aid,Alpine,Boulder,Ice,Mixed,Sport,TR,Trad,...,area_World WallaAreas,area_Write-off RockaAreas,area_Wu Wei WallaAreas,area_Wu-Tang WallaAreas,area_XY Crag (Chromosome Cracks)aAreas,area_Zappa WallaAreas,area_Zeke's Trail BouldersaAreas,area_Zeke's WallaAreas,area_Zig Zag WallaAreas,num_reviews
0,0,30(1) Feet of Pleasure,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,1,41 Feet of Pain,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,2,5.5 Crack,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,4
3,3,5.6 crack,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,4
4,4,5.6 Dihedral,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,4


In [18]:
# convert colnames to utf-8
col_list = []
for col in routes.columns:
    col_list.append(col.encode('utf-8'))
routes.columns = col_list

In [19]:
routes.drop('name', axis=1).to_csv("routes_df.csv", sep='\t')

### prep user info

In [20]:
db = client.users
users_df = dataframe_from_collection(db.users)
users_df.head().T

,0,1,2,3,4
_id,58d1912840b4417fa4b16db4,58d1912840b4417fa4b16db5,58d1912840b4417fa4b16db6,58d1912840b4417fa4b16db7,58d1912840b4417fa4b16db8
age,NaN,NaN,NaN,33,NaN
aid_follows,NaN,NaN,C5,NaN,NaN
aid_leads,NaN,NaN,C3,NaN,NaN
city,NaN,Seattle,Hawaii,Corvallis,NaN
compliments,0,1,10,0,0
favorite_climbs,leading trad is rad,"high mountain woody, fridge left, easy in an ...",none specified,outer space,none specified
female,NaN,0,0,0,NaN
ice_follows,NaN,NaN,NaN,NaN,NaN
ice_leads,NaN,NaN,NaN,NaN,NaN


### make id dataframe

In [21]:
users_df_id = users_df[['name', 'id']]

In [22]:
users_df_new = users_df.drop(['name', '_id', 'other_interests', 'id', 'last_vist', 'favorite_climbs'], axis=1)
users_df_new.head(10).T

,0,1,2,3,4,5,6,7,8,9
age,NaN,NaN,NaN,33,NaN,44,NaN,25,26,NaN
aid_follows,NaN,NaN,C5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aid_leads,NaN,NaN,C3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,NaN,Seattle,Hawaii,Corvallis,NaN,Fort Thomas,San Diego,albuquerque,Seattle,Redmond
compliments,0,1,10,0,0,0,2,0,0,0
female,NaN,0,0,0,NaN,0,1,1,0,0
ice_follows,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI4,NaN
ice_leads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WI3,NaN
likes_gym,0,1,1,1,0,1,0,1,0,1
likes_sport,0,1,1,1,0,1,0,1,0,1


In [23]:
users_df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Data columns (total 20 columns):
age              903 non-null float64
aid_follows      217 non-null object
aid_leads        217 non-null object
city             1155 non-null object
compliments      2069 non-null int64
female           1260 non-null float64
ice_follows      314 non-null object
ice_leads        314 non-null object
likes_gym        2069 non-null int64
likes_sport      2069 non-null int64
likes_tr         2069 non-null int64
likes_trad       2069 non-null int64
member_since     2069 non-null object
point_rank       1156 non-null float64
sport_follows    1036 non-null object
sport_lead       1036 non-null object
state            1133 non-null object
total_points     2069 non-null int64
trad_follows     998 non-null object
trad_lead        998 non-null object
dtypes: float64(3), int64(6), object(11)
memory usage: 323.4+ KB


### fill nulls

In [24]:
# fill age with mean age
users_df_new['age'] = users_df_new['age'].fillna(users_df_new['age'].mean())

In [25]:
# fix point rank
users_df_new['point_rank'] = users_df_new['point_rank'].fillna(users_df_new['point_rank'].mean())

In [26]:
# fix datetime
users_df_new['member_since'] = pd.to_datetime(users_df_new['member_since'])
users_df_new.head().T

,0,1,2,3,4
age,33.2824,33.2824,33.2824,33,33.2824
aid_follows,NaN,NaN,C5,NaN,NaN
aid_leads,NaN,NaN,C3,NaN,NaN
city,NaN,Seattle,Hawaii,Corvallis,NaN
compliments,0,1,10,0,0
female,NaN,0,0,0,NaN
ice_follows,NaN,NaN,NaN,NaN,NaN
ice_leads,NaN,NaN,NaN,NaN,NaN
likes_gym,0,1,1,1,0
likes_sport,0,1,1,1,0


### time to make dummies

In [27]:
dummy_list1 = [u'aid_follows',
             u'aid_leads',
             u'ice_follows',
             u'ice_leads',
             u'sport_follows',
             u'sport_lead',
             u'trad_follows',
             u'trad_lead']
dummy_list2 = [u'city',u'state',]

In [28]:
df_subsets = users_df_new[dummy_list1]
dummy_df1 = pd.get_dummies(df_subsets)

In [29]:
df_subsets = users_df_new[dummy_list2]
dummy_df2 = pd.get_dummies(df_subsets)

In [30]:
dummy_df3 = pd.get_dummies(users_df_new['female']).rename( columns={0.0: 'male', 1.0: 'female'})

In [31]:
users_df_new.head().T
concat_list = [users_df_new.drop(dummy_list1+dummy_list2+['female'], axis=1), dummy_df1, dummy_df2, dummy_df3]
x_users = pd.concat(concat_list, axis=1)

In [32]:
x_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2069 entries, 0 to 2068
Columns: 599 entries, age to female
dtypes: datetime64[ns](1), float64(2), int64(6), uint8(590)
memory usage: 1.3 MB


In [33]:
x_users.isnull().values.any()

False

### save to csv

In [34]:
users = users_df_id[['id', 'name']].join(x_users)
users.to_csv("users_df.csv", sep='\t')

### Combine x_users and x_routes based on the utility matrix

In [35]:
# get list of indices
route_ind_list = ratings_df['route_id'].tolist()
user_ind_list = ratings_df['user_id'].tolist()

In [36]:
left = x_users.iloc[user_ind_list, :].reset_index().drop('index', axis=1)
right = x_routes.iloc[route_ind_list, :].reset_index().drop('index', axis=1)
data_df = left.join(right)

In [37]:
data_df.isnull().values.any()

False

In [38]:
model_df = data_df.join(ratings_df)
model_df.head()

,age,compliments,likes_gym,likes_sport,likes_tr,likes_trad,member_since,point_rank,total_points,aid_follows_ C0,...,area_Wu Wei WallaAreas,area_Wu-Tang WallaAreas,area_XY Crag (Chromosome Cracks)aAreas,area_Zappa WallaAreas,area_Zeke's Trail BouldersaAreas,area_Zeke's WallaAreas,area_Zig Zag WallaAreas,rating,route_id,user_id
0,35.000000,40,1,1,1,1,2008-06-29,611.000000,1100,1,...,0,0,0,0,0,0,0,1,0,36
1,35.000000,40,1,1,1,1,2008-06-29,611.000000,1100,1,...,0,0,0,0,0,0,0,1,1,36
2,29.000000,4,1,1,0,1,2011-05-28,172.000000,2945,0,...,0,0,0,0,0,0,0,1,2,394
3,27.000000,0,1,1,1,1,2012-04-16,1053.000000,656,0,...,0,0,0,0,0,0,0,1,2,1972
4,33.282392,0,0,0,0,0,2015-04-20,6388.894464,0,0,...,0,0,0,0,0,0,0,1,2,1625


In [39]:
model_df.isnull().values.any()

False

In [40]:
# convert colnames to utf-8
col_list = []
for col in model_df.columns:
    col_list.append(col.encode('utf-8'))
model_df.columns = col_list

### save as csv because mongodb doesn't like '.'  in dict key

In [41]:
model_df.to_csv("model_df.csv", sep='\t')

### add number of reviews

In [42]:
# load data frame from csv
data_df = pd.read_csv("model_df.csv", sep='\t')

In [43]:
data_df.head()

,Unnamed: 0,age,compliments,likes_gym,likes_sport,likes_tr,likes_trad,member_since,point_rank,total_points,...,area_Wu Wei WallaAreas,area_Wu-Tang WallaAreas,area_XY Crag (Chromosome Cracks)aAreas,area_Zappa WallaAreas,area_Zeke's Trail BouldersaAreas,area_Zeke's WallaAreas,area_Zig Zag WallaAreas,rating,route_id,user_id
0,0,35.000000,40,1,1,1,1,2008-06-29,611.000000,1100,...,0,0,0,0,0,0,0,1,0,36
1,1,35.000000,40,1,1,1,1,2008-06-29,611.000000,1100,...,0,0,0,0,0,0,0,1,1,36
2,2,29.000000,4,1,1,0,1,2011-05-28,172.000000,2945,...,0,0,0,0,0,0,0,1,2,394
3,3,27.000000,0,1,1,1,1,2012-04-16,1053.000000,656,...,0,0,0,0,0,0,0,1,2,1972
4,4,33.282392,0,0,0,0,0,2015-04-20,6388.894464,0,...,0,0,0,0,0,0,0,1,2,1625


In [44]:
data_df = data_df.drop('Unnamed: 0', axis=1)

In [45]:
data_df['num_reviews'] = data_df.groupby(['route_id'])['user_id'].transform('count')
data_df.head(50)

,age,compliments,likes_gym,likes_sport,likes_tr,likes_trad,member_since,point_rank,total_points,aid_follows_ C0,...,area_Wu-Tang WallaAreas,area_XY Crag (Chromosome Cracks)aAreas,area_Zappa WallaAreas,area_Zeke's Trail BouldersaAreas,area_Zeke's WallaAreas,area_Zig Zag WallaAreas,rating,route_id,user_id,num_reviews
0,35.000000,40,1,1,1,1,2008-06-29,611.000000,1100,1,...,0,0,0,0,0,0,1,0,36,1
1,35.000000,40,1,1,1,1,2008-06-29,611.000000,1100,1,...,0,0,0,0,0,0,1,1,36,1
2,29.000000,4,1,1,0,1,2011-05-28,172.000000,2945,0,...,0,0,0,0,0,0,1,2,394,4
3,27.000000,0,1,1,1,1,2012-04-16,1053.000000,656,0,...,0,0,0,0,0,0,1,2,1972,4
4,33.282392,0,0,0,0,0,2015-04-20,6388.894464,0,0,...,0,0,0,0,0,0,1,2,1625,4
5,14.000000,0,0,1,1,1,2016-06-21,7902.000000,31,0,...,0,0,0,0,0,0,1,2,1522,4
6,33.282392,0,0,0,0,0,2016-03-09,6388.894464,0,0,...,0,0,0,0,0,0,3,3,233,34
7,30.000000,1,1,1,1,1,2014-09-22,2899.000000,186,0,...,0,0,0,0,0,0,3,3,1716,34
8,34.000000,20,1,1,1,1,2014-07-14,1217.000000,551,0,...,0,0,0,0,0,0,3,3,1399,34
9,33.282392,8,1,1,1,1,2009-09-15,238.000000,2385,0,...,0,0,0,0,0,0,2,3,1935,34


In [46]:
data_df.to_csv("model_df.csv", sep='\t')

In [47]:
model_df[model_df['user_id'] == 560]['route_id']

1610      199
2825      370
5765      684
8750      956
11226    1200
11924    1244
13621    1395
16397    1705
19840    2086
21128    2237
22236    2417
24016    2663
26972    3128
28405    3351
29000    3474
Name: route_id, dtype: int64